<a href="https://colab.research.google.com/github/Vaycold/Python_DL/blob/main/Text_Classification/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>